## Weather Data Files Reformatting

In [1]:
from datetime import date
import time
import pandas as pd
import os

#### first polish

In [2]:
def shape_df(file):
    df = pd.read_csv(file) #read files
    df = df.loc[:, ['ZONE_CODE','LPT_START_DT', 'WEATHER_CONCEPT_CODE','WEATHER_VALUE']] #only use these columns
    df.columns = ['Zone', 'DT', 'Type', 'Value'] #change a easy name
    
    df['DT'] = pd.to_datetime(df['DT']) #change time format
    
    df['Date'] =  df['DT'].dt.date # get date
    df["Time"] = df['DT'].dt.time  #get time
    
    del df['DT'] #drop DT column
    
    df = df.sort_values(by = ['Zone','Type','Date','Time']).reset_index(drop = True) #sort data
    
    return df

###### check how the data look like by zones and by types

In [18]:
files_dir = 'C:/Users/zhongj/Desktop/WeatherData' # change the directory
os.chdir(files_dir)

df = shape_df('weather_2020.csv')

In [21]:
Zone_list = list(df['Zone'].unique()) # unique Zonal names
Type_list = list(df['Type'].unique()) # unique Types names

In [22]:
for j in Zone_list:
    print(df[df['Zone']==j])

      Zone Type    Value        Date      Time
0        A  CLC   71.599  2020-01-01  00:00:00
1        A  CLC  100.000  2020-01-01  01:00:00
2        A  CLC  100.000  2020-01-01  02:00:00
3        A  CLC  100.000  2020-01-01  03:00:00
4        A  CLC  100.000  2020-01-01  04:00:00
...    ...  ...      ...         ...       ...
87835    A  WSP    4.820  2020-12-31  19:00:00
87836    A  WSP    6.360  2020-12-31  20:00:00
87837    A  WSP    5.880  2020-12-31  21:00:00
87838    A  WSP    5.870  2020-12-31  22:00:00
87839    A  WSP    1.890  2020-12-31  23:00:00

[87840 rows x 5 columns]
       Zone Type    Value        Date      Time
87840     B  CLC   71.596  2020-01-01  00:00:00
87841     B  CLC   74.602  2020-01-01  01:00:00
87842     B  CLC   74.602  2020-01-01  02:00:00
87843     B  CLC   74.602  2020-01-01  03:00:00
87844     B  CLC  100.000  2020-01-01  04:00:00
...     ...  ...      ...         ...       ...
175675    B  WSP    4.910  2020-12-31  19:00:00
175676    B  WSP    8.230 

In [33]:
for j in Type_list: 
    print(df[df['Type']==j])

       Zone Type    Value        Date      Time
0         A  CLC   71.599  2020-01-01  00:00:00
1         A  CLC  100.000  2020-01-01  01:00:00
2         A  CLC  100.000  2020-01-01  02:00:00
3         A  CLC  100.000  2020-01-01  03:00:00
4         A  CLC  100.000  2020-01-01  04:00:00
...     ...  ...      ...         ...       ...
975019    S  CLC   70.160  2020-12-31  19:00:00
975020    S  CLC   72.850  2020-12-31  20:00:00
975021    S  CLC   57.420  2020-12-31  21:00:00
975022    S  CLC   41.120  2020-12-31  22:00:00
975023    S  CLC   35.270  2020-12-31  23:00:00

[105408 rows x 5 columns]
       Zone   Type    Value        Date      Time
8784      A  DBT60  31.1856  2020-01-01  00:00:00
8785      A  DBT60  31.1323  2020-01-01  01:00:00
8786      A  DBT60  31.1323  2020-01-01  02:00:00
8787      A  DBT60  30.1185  2020-01-01  03:00:00
8788      A  DBT60  30.1323  2020-01-01  04:00:00
...     ...    ...      ...         ...       ...
983803    S  DBT60  36.4550  2020-12-31  19:00:

###### check what time/date is missing or duplicate:

In [24]:
Time_list = list(df['Time'].unique())
Date_list = list(df['Date'].unique())

In [39]:
for j in Date_list:
    a = list(df[(df['Type']=='WSP')&(df['Zone']== 'K')]['Date']).count(j)
    if a < 24:
        print(j, a)

2020-03-08 23


In [28]:
for j in Time_list:
    a = list(df[(df['Type']=='WSP')&(df['Zone']== 'A')]['Time']).count(j)
    print(j, a)
### from here we know that 1am has a dupicated row

00:00:00 366
01:00:00 367
02:00:00 365
03:00:00 366
04:00:00 366
05:00:00 366
06:00:00 366
07:00:00 366
08:00:00 366
09:00:00 366
10:00:00 366
11:00:00 366
12:00:00 366
13:00:00 366
14:00:00 366
15:00:00 366
16:00:00 366
17:00:00 366
18:00:00 366
19:00:00 366
20:00:00 366
21:00:00 366
22:00:00 366
23:00:00 366


In [29]:
## check duplicated raws:

import datetime

Date_list = list(df['Date'].unique())

for j in Date_list:
    a = list(df[(df['Type']=='WSP')&(df['Zone']== 'A')&(df['Time'] == datetime.time(1,0))]['Date']).count(j)
    if a > 1:
        print(j, a)

### from here we know 11.05 1:00:00 has a duplicated raw

2020-11-01 2


In [38]:
df[ (df['Type']=='GHI60') & (df['Zone']== 'G') & (df['Time'] == datetime.time(1,0)) & (df['Date'] == date(2020,11,1)) ]

,Zone,Type,Value,Date,Time


In [30]:
## check duplicated raws:

import datetime

Date_list = list(df['Date'].unique())

for j in Date_list:
    a = list(df[(df['Type']=='WSP')&(df['Zone']== 'A')&(df['Time'] == datetime.time(23,0))]['Date']).count(j)
    if a > 1:
        print(j, a)

### from here we know 11-05-2017 1:00:00 has a duplicated raw
### and 11-01-2020 1:00:00 has a duplicated raw

#### add missing rows and delet the dupicates 

##### 2017: 
    dupicate: 11-05-2017 1:00:00
    
    missing: 

    
         GHI60, SSM, THI, WCI, WDR: missing from 01-01-2017 ~ 10-28-2017
         
         CLC, DEW, WET, WSP, DBT60:
                 2017-01-09 2017-01-25 2017-02-04, 2017-02-09,
                 2017-03-04, 2017-03-12, 2017-05-03, 2017-08-05,
                 0217-08-14, 2017-09-03, 2017-09-06, 2017-09-10
                 
                 at
                 
                 2:00 ~ 14:00 (hourly interval)
###### 2020:

    dupicate: 11-01-2020 1:00:00
    
    missing: 2020-03-08 at 2:00:00
    

In [48]:
def add_missing(df): 
    
    # Delete duplicated raws:
    df = df.drop_duplicates(['Zone','Type','Date','Time', 'Value']).reset_index(drop = True)
    Zone_list = list(df['Zone'].unique()) # unique Zonal names
    Type_list = list(df['Type'].unique()) # unique Types names
    
    #2017:
    if len(df.loc[(df['Date'] <= date(2017,12,31))]) > 0:

        # Add missing GHI60, SSM, THI, WCI, WDR raws: 
        for Zone in Zone_list:
            for j in Type_list:
                if len(df.loc[(df['Date'] < date(2017,10,29)) & (df['Type']== j) & (df['Zone']== Zone)]) == 0:
                    df_add_raw = pd.DataFrame(columns = df.columns) # create a dateframe
                    idx = pd.date_range(start = '2017-01-01', end = '2017-10-28 23:00:00', freq = 'H')
                    df_add_raw['Zone'] = ["%s"%(Zone)]*len(idx)
                    df_add_raw['Type'] = ["%s"%(j)]*len(idx) 
                    df_add_raw['Date'] = idx.date
                    df_add_raw['Time'] = idx.time

                    df = df.append(df_add_raw, ignore_index = True)

        df = df.sort_values(by = ['Zone','Type','Date','Time']).reset_index(drop = True)

        # Add missing CLC, DBT60, DEW, WET, WSP raw: 
        Time_list = list(df['Time'].unique())
        Date_list = list(df['Date'].unique())
        type_list = ['CLC','DBT60','DEW','WET','WSP'] 

        # Find dates that have missing times:
        miss_date = []
        for j in Date_list:
            a = list(df[(df['Type']=='WSP')&(df['Zone']== 'A')]['Date']).count(j)
            if a < 24:
                miss_date.append(j)

        # add raws:
        df_add_raw = pd.DataFrame(columns = df.columns)
        for Zone in Zone_list:
            for j in Type_list:
                df_cut = df[(df['Type']==j) & (df['Zone']== Zone)] 

                for i in miss_date:
                    time_list = list(df_cut[(df['Date'] == i)]['Time'].unique()) # times that occur in that day
                    miss_time = list(set(Time_list) - set(time_list)) #find what times are missing in that day

                    #add the time:
                    for t in miss_time:                
                        df_add_raw.loc[df_add_raw.shape[0]] = dict(zip(df_add_raw.columns, 
                                                                       ("%s"%(Zone), "%s"%(j), None, 
                                                                        i, t)))

        df = df.append(df_add_raw, ignore_index = True)
        df = df.sort_values(by = ['Zone','Type','Date','Time']).reset_index(drop = True)
        
        return df
    
    # 2020
    if len(df.loc[(df['Date'] >= date(2020,1,1))]) > 0:
        # add raws:
        for Zone in Zone_list:
            for j in Type_list:
                df.loc[df.shape[0]] = dict(zip(df.columns, 
                                               ("%s"%(Zone), "%s"%(j), None,
                                                date(2020,3,8), datetime.time(2,0))))

        df = df.sort_values(by = ['Zone','Type','Date','Time']).reset_index(drop = True)

        return df

    # 2018, 2019
    else:
        return df

#### seperate/add columns by Types; and seperate files by Zones

In [50]:
import os

def seperate_types_files(file, files_dir): # input is a file, and files_dir: where you want to store the new files
    # get the file name:
    filename = os.path.basename(file)
    filename = os.path.splitext(filename)[0] 
    
    df =  shape_df(file) # use function above to get the modified dataframe
    df = add_missing(df) # use function above to get the modified dataframe
    
    Zone_list = list(df['Zone'].unique()) # unique Zonal names
    Zone_list.sort()
    Type_list = list(df['Type'].unique()) # unique Types names
    Type_list.sort()
    
    
    for i in Zone_list:
        
        os.chdir(files_dir) # change the path to the directory
        
        # if the zonal file is not in the directory:
        if os.path.exists("%s_%s"%(i, filename)):
            pass
        
        else:
            df_add = pd.DataFrame(columns = Type_list) # create a dateframe
            file_cut = df[df['Zone']==i] # Seperate Zones

            # Seperate Types to columes, and add them to a new dataframe:
            for j in Type_list: 
                type_cut = file_cut.loc[file_cut['Type']==j]  
                df_add[j] = type_cut['Value'].values

            file_cut = file_cut.drop_duplicates(['Date','Time']).reset_index(drop = True) # Delete duplicated raws

            del file_cut['Type'] # drop Types column
            del file_cut['Value'] # drop READING column

            final = pd.concat([file_cut,df_add], axis = 1, ignore_index=True) # combine the new dataframe to old one

            # Rename the final dataframe:
                ## underscore means its weather data
            final.columns =  ['Zone', 'Date','Time' ,
                              '_CLC', '_DBT60', '_DEW', '_GHI60','_SSM', 
                              '_THI', '_WCI', '_WDR', '_WET', '_WSP'] 


            final.to_excel(r"%s_%s.xlsx"%(i, filename), 'sheet1', index=False) # create a excel file


#### all files

In [51]:
files_dir_old = 'C:/Users/zhongj/Desktop/WeatherData' # change the directory

files_dir_new = "C:/Users/zhongj/Desktop/WeatherData_Edit" # change the directory


for filename in os.listdir(files_dir_old):
    file_path = os.path.join(files_dir_old, filename)
    seperate_types_files(file_path, files_dir_new)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
